<a href="https://colab.research.google.com/github/PotatoSloth/Duran-Duran-Returns/blob/master/0128_scaling_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Colaboratory 실행 코드 

In [0]:
# 처음 실행할 때 한 번만! 
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools

!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null

!apt-get update -qq 2>&1 > /dev/null

!apt-get -y install -qq google-drive-ocamlfuse fuse

from google.colab import auth

auth.authenticate_user()

from oauth2client.client import GoogleCredentials

creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [4]:
!mkdir -p drive

!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [5]:
cd drive

/content/drive


## Import Packages 

In [0]:
# -*- coding: utf_8 -*-
# Linear algebra 

import numpy as np 

# Data processing 
import pandas as pd 

# import XGB estimator 
from xgboost.sklearn import XGBRegressor 
import xgboost as xgb
# Metrics for root mean squared error
from sklearn.metrics import mean_squared_error
from math import sqrt
from tqdm import tqdm

In [0]:
train= pd.read_csv('./Colab Notebooks/data/train.csv')

In [8]:
train.info()
# total_parking 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 25 columns):
key                                   1601458 non-null int64
apartment_id                          1601458 non-null int64
city                                  1601458 non-null int64
transaction_year_month                1601458 non-null int64
transaction_date                      1601458 non-null object
year_of_completion                    1601458 non-null int64
exclusive_use_area                    1601458 non-null float64
floor                                 1601458 non-null int64
latitude                              1601458 non-null float64
longitude                             1601458 non-null float64
address_by_law                        1601458 non-null int64
total_parking_capacity_in_site        1509645 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_site

## Data preprocessing 

### dropping 

In [0]:
drop = ['key','apartment_id','heat_type','heat_fuel','front_door_structure']
train.drop(drop, axis = 1, inplace = True)
# test.drop(drop, axis =1, inplace = True)
# 결측치가 있는 것도 정상적으로 입력값으로 받을 것인가? XGBoost


In [10]:
train.sample(n= 20)

,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
906483,1,201405,21~31,1992,58.4600,2,37.652823,127.075925,1135010600,2328.0,2328,18,15.0,15.0,2917,80.31,528,2.0,1.0,270000000
1260688,1,201606,21~30,1988,74.5800,10,37.635445,127.064240,1135010400,360.0,1980,12,15.0,15.0,20496,89.31,180,3.0,2.0,360000000
1109212,1,201507,1~10,1988,136.3250,2,37.489072,127.115487,1171010800,4494.0,4494,56,15.0,13.0,4038,158.84,1416,4.0,2.0,900000000
1484245,0,201710,21~31,2008,215.6400,48,35.177781,129.124461,2635010400,1850.0,629,3,60.0,21.0,94281,285.23,36,4.0,3.0,1125000000
574552,0,201101,1~10,1988,43.7200,2,35.201703,129.132545,2635010300,NaN,200,5,5.0,5.0,54102,53.17,80,2.0,1.0,63000000
1429913,1,201706,1~10,1984,109.0700,4,37.535402,127.148240,1174010600,311.0,354,4,15.0,12.0,3374,129.31,180,4.0,2.0,600000000
298828,1,200806,11~20,1988,136.3250,9,37.489072,127.115487,1171010800,4494.0,4494,56,15.0,13.0,4038,158.84,1416,4.0,2.0,1170000000
447673,1,200911,11~20,1999,59.3400,18,37.468343,126.893293,1154510200,244.0,245,1,19.0,6.0,6774,80.23,104,3.0,1.0,265000000
474059,1,201002,11~20,1990,59.2200,3,37.581320,127.095950,1126010100,565.0,565,4,15.0,8.0,147114,78.14,181,3.0,1.0,200000000
1307398,1,201609,1~10,1983,118.6200,6,37.516425,126.892795,1156012100,189.0,270,2,15.0,15.0,7392,138.50,60,4.0,2.0,650000000


In [0]:
all_ = train
# train 1601458 
# test start from 1601458 

In [12]:
all_.head()

,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
0,1,200601,11~20,1999,47.43,6,37.585965,127.000231,1111017100,163.0,136,1,8.0,4.0,91120,65.63,46,1.0,1.0,215000000
1,1,200601,11~20,1999,44.37,8,37.585965,127.000231,1111017100,163.0,136,1,8.0,4.0,91119,61.39,10,2.0,1.0,200000000
2,1,200601,11~20,1992,54.70,8,37.580511,127.014016,1111017400,902.0,585,5,14.0,9.0,8430,72.36,201,2.0,1.0,168000000
3,1,200601,11~20,1993,64.66,11,37.580324,127.011788,1111017400,902.0,919,7,15.0,11.0,5839,87.30,284,2.0,1.0,165000000
4,1,200601,11~20,1993,106.62,7,37.580324,127.011788,1111017400,902.0,919,7,15.0,11.0,5836,127.74,112,4.0,2.0,280000000


### Feature Engineering 

In [0]:
trans_date = {'1~10':'월초','11~20':'월중','21~28':'월말', '21~29':'월말','21~30':'월말', '21~31':'월말'}
all_['transaction_date'] = [trans_date[i] for i in all_['transaction_date']]

In [14]:
all_.head()

,city,transaction_year_month,transaction_date,year_of_completion,exclusive_use_area,floor,latitude,longitude,address_by_law,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,room_id,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price
0,1,200601,월중,1999,47.43,6,37.585965,127.000231,1111017100,163.0,136,1,8.0,4.0,91120,65.63,46,1.0,1.0,215000000
1,1,200601,월중,1999,44.37,8,37.585965,127.000231,1111017100,163.0,136,1,8.0,4.0,91119,61.39,10,2.0,1.0,200000000
2,1,200601,월중,1992,54.70,8,37.580511,127.014016,1111017400,902.0,585,5,14.0,9.0,8430,72.36,201,2.0,1.0,168000000
3,1,200601,월중,1993,64.66,11,37.580324,127.011788,1111017400,902.0,919,7,15.0,11.0,5839,87.30,284,2.0,1.0,165000000
4,1,200601,월중,1993,106.62,7,37.580324,127.011788,1111017400,902.0,919,7,15.0,11.0,5836,127.74,112,4.0,2.0,280000000


In [0]:
# 날짜 데이터를 숫자로 변환 
# 월 단위 숫자로 변환. 
# train['transaction_year_month'] = train['transaction_year_month'].astype(str)
def date_to_int(df):
    df['transaction_year_month'] = df['transaction_year_month'].astype(str)
    Y = [int(a[:4]) for a in df['transaction_year_month']]
    M = [int(a[4:]) for a in df['transaction_year_month']]
    int_date=[]
    for i in range(len(Y)):
        int_date.append((Y[i] - 2006) * 12 +M[i])
    df['transaction_int_date'] = int_date
    
    return df
# int(train.transaction_year_month[0][:4])
# train['transaction_int_date'] = train['transaction_year_month'].map(date_to_int).astype(np.int8)


In [16]:
test = date_to_int(all_)
test.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 21 columns):
city                                  1601458 non-null int64
transaction_year_month                1601458 non-null object
transaction_date                      1601458 non-null object
year_of_completion                    1601458 non-null int64
exclusive_use_area                    1601458 non-null float64
floor                                 1601458 non-null int64
latitude                              1601458 non-null float64
longitude                             1601458 non-null float64
address_by_law                        1601458 non-null int64
total_parking_capacity_in_site        1509645 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_sites             1601449 non-null float64
lowest_building_in_sites              1601449 non-null float64
room_id            

#### Address

In [0]:
test['address_by_law'] = test['address_by_law'].astype(str)
test['district'] = test['address_by_law'].apply(lambda x: x[:5])

In [18]:
test=test.drop(columns = ['address_by_law','transaction_year_month','exclusive_use_area','latitude','longitude','room_id'])
test.head()

,city,transaction_date,year_of_completion,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price,transaction_int_date,district
0,1,월중,1999,6,163.0,136,1,8.0,4.0,65.63,46,1.0,1.0,215000000,1,11110
1,1,월중,1999,8,163.0,136,1,8.0,4.0,61.39,10,2.0,1.0,200000000,1,11110
2,1,월중,1992,8,902.0,585,5,14.0,9.0,72.36,201,2.0,1.0,168000000,1,11110
3,1,월중,1993,11,902.0,919,7,15.0,11.0,87.30,284,2.0,1.0,165000000,1,11110
4,1,월중,1993,7,902.0,919,7,15.0,11.0,127.74,112,4.0,2.0,280000000,1,11110


In [0]:
# test
list4 = []
s_center = ['11110','11140','11170']
s_eastnorth = ['11200','11215','11230','11260','11290','11305','11320','11350'] 
s_westnorth = ['11380','11410','11440'] 
s_westsouth = ['11470','11500','11530','11545','11560','11590','11620']
s_eastsouth = ['11650','11680','11710','11740']
p_center = ['26110','26140','26170','26200','26230','26290','26470','26500']
p_east = ['26350','26410','26260','26710']
p_west=['26320','26440','26530','26380']

for i in test['district']:
    if i in s_center:
        list4.append('도심권')
    elif i in s_eastnorth:
        list4.append('동북권')
    elif i in s_westnorth:
        list4.append('서북권')
    elif i in s_westsouth:
        list4.append('서남권')
    elif i in s_eastsouth:
        list4.append('동남권')
    elif i in p_center:
        list4.append('중부산권')
    elif i in p_east:
        list4.append('동부산권')
    else:
        list4.append('서부산권')

In [20]:
test['district_group'] = list4
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 17 columns):
city                                  1601458 non-null int64
transaction_date                      1601458 non-null object
year_of_completion                    1601458 non-null int64
floor                                 1601458 non-null int64
total_parking_capacity_in_site        1509645 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_sites             1601449 non-null float64
lowest_building_in_sites              1601449 non-null float64
supply_area                           1601458 non-null float64
total_household_count_of_area_type    1601458 non-null int64
room_count                            1600767 non-null float64
bathroom_count                        1600767 non-null float64
transaction_real_price                1601458 non-null int64
transaction_int_date

In [21]:
test.head()


,city,transaction_date,year_of_completion,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price,transaction_int_date,district,district_group
0,1,월중,1999,6,163.0,136,1,8.0,4.0,65.63,46,1.0,1.0,215000000,1,11110,도심권
1,1,월중,1999,8,163.0,136,1,8.0,4.0,61.39,10,2.0,1.0,200000000,1,11110,도심권
2,1,월중,1992,8,902.0,585,5,14.0,9.0,72.36,201,2.0,1.0,168000000,1,11110,도심권
3,1,월중,1993,11,902.0,919,7,15.0,11.0,87.30,284,2.0,1.0,165000000,1,11110,도심권
4,1,월중,1993,7,902.0,919,7,15.0,11.0,127.74,112,4.0,2.0,280000000,1,11110,도심권


In [22]:
test['city'] = test['city'].apply(lambda x :'seoul' if x == 1 else 'busan' )
test['floor'] = test['floor'].apply(lambda x : '저층' if x < 8 else '중층' if 8<x<18 else '고층' if 18<=x<=30 else '초고층') 

test.sample(n=10)

,city,transaction_date,year_of_completion,floor,total_parking_capacity_in_site,total_household_count_in_sites,apartment_building_count_in_sites,tallest_building_in_sites,lowest_building_in_sites,supply_area,total_household_count_of_area_type,room_count,bathroom_count,transaction_real_price,transaction_int_date,district,district_group
1104741,seoul,월초,1995,중층,2488.0,1244,28,12.0,5.0,151.72,672,4.0,2.0,560000000,115,11260,동북권
1578627,seoul,월중,2010,저층,855.0,472,22,15.0,4.0,124.10,0,3.0,2.0,700000000,152,11380,서북권
1545173,busan,월초,1998,중층,481.0,799,6,25.0,20.0,79.54,40,3.0,1.0,260000000,147,26350,동부산권
414819,seoul,월초,1989,저층,676.0,2265,26,15.0,5.0,77.11,150,2.0,1.0,265000000,44,11350,동북권
185448,seoul,월말,1971,저층,NaN,108,3,6.0,6.0,54.96,108,2.0,1.0,130000000,18,11620,서남권
883084,seoul,월초,2008,저층,7712.0,5678,72,34.0,17.0,84.75,1150,3.0,2.0,700000000,99,11710,동남권
489709,seoul,월말,2005,중층,993.0,689,11,20.0,11.0,107.97,280,3.0,2.0,410000000,51,11290,동북권
1536065,seoul,월말,1993,저층,1033.0,990,11,15.0,15.0,103.29,990,3.0,2.0,600000000,146,11500,서남권
1363275,seoul,월말,2001,저층,2776.0,2298,27,27.0,19.0,82.54,384,3.0,1.0,380000000,133,11530,서남권
605778,seoul,월초,2011,중층,967.0,889,13,15.0,7.0,111.15,56,3.0,2.0,399460000,64,11710,동남권


In [23]:
test.drop('district', axis= 1, inplace= True)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601458 entries, 0 to 1601457
Data columns (total 17 columns):
city                                  1601458 non-null object
transaction_date                      1601458 non-null object
year_of_completion                    1601458 non-null int64
floor                                 1601458 non-null object
total_parking_capacity_in_site        1509645 non-null float64
total_household_count_in_sites        1601458 non-null int64
apartment_building_count_in_sites     1601458 non-null int64
tallest_building_in_sites             1601449 non-null float64
lowest_building_in_sites              1601449 non-null float64
supply_area                           1601458 non-null float64
total_household_count_of_area_type    1601458 non-null int64
room_count                            1600767 non-null float64
bathroom_count                        1600767 non-null float64
transaction_real_price                1601458 non-null int64
transaction_int_da

In [0]:
# Data preprocess and feature engineering 
def apply_transform(test):
    features = ['year_of_completion','total_household_count_in_sites','apartment_building_count_in_sites','supply_area','total_household_count_of_area_type','transaction_int_date']
    
    
    # 주차장 결측치를 평균값으로 대체한다. 
    test['total_parking_capacity_in_site'] = test['total_parking_capacity_in_site'].fillna(test['total_parking_capacity_in_site'].mean()).astype(np.int16)
    features.append('total_parking_capacity_in_site')
    
    test['room_count'] = test['room_count'].fillna(test['room_count'].mean()).astype(np.int16)
    features.append('room_count')
    test['bathroom_count'] = test['bathroom_count'].fillna(test['bathroom_count'].mean()).astype(np.int16)
    features.append('bathroom_count')
    test['tallest_building_in_sites'] = test['tallest_building_in_sites'].fillna(0.0).astype(np.int16)
    features.append('tallest_building_in_sites')
    test['lowest_building_in_sites'] = test['lowest_building_in_sites'].fillna(0.0).astype(np.int16)
    features.append('lowest_building_in_sites')
#     target = test.transaction_real_price
#     test.drop(['transaction_real_price'],axis=1, inplace = True)

#     # one-hot encoding 수행()
    test1=pd.get_dummies(test['transaction_date'],prefix='date')
    features.append('date_월초')
    features.append('date_월중')
    features.append('date_월말')
    test2=pd.get_dummies(test['city'],prefix='city')
    features.append('city_seoul')
    features.append('city_busan')
    test3=pd.get_dummies(test['floor'],prefix='floor')
    features.append('floor_저층')
    features.append('floor_중층')
    features.append('floor_고층')
    features.append('floor_초고층')
    test4=pd.get_dummies(test['district_group'],prefix='district')
    features.append('district_도심권')
    features.append('district_동북권')
    features.append('district_서북권')
    features.append('district_서남권')
    features.append('district_동남권')
    features.append('district_중부산권')
    features.append('district_서부산권')
    features.append('district_동부산권')
    test=pd.concat([test,test1,test2,test3,test4],axis=1) 
    test=test.drop(columns=['city','floor','district_group','transaction_date'])
    return test, tuple(features)
    

In [0]:
test, features = apply_transform(test) 
# test는 주요 변수를 포함한 훈련/테스트 데이터 통합 데이터프레임 
# features는 학습에 사용하기 위한 변수를 기록한 튜플

In [38]:
# train_y=test['transaction_real_price']
# test.drop('transaction_real_price', axis = 1, inplace=True)
train_y

0          215000000
1          200000000
2          168000000
3          165000000
4          280000000
5          415000000
6          267000000
7          415000000
8          310000000
9          319000000
10         246000000
11         352000000
12         667500000
13         667500000
14         360000000
15         440000000
16         280000000
17         708000000
18         270000000
19         600000000
20         600000000
21         289000000
22         420000000
23         396000000
24         287000000
25         287000000
26         242500000
27         242500000
28         429000000
29         280000000
             ...    
1601428    292000000
1601429    475000000
1601430    600000000
1601431    447000000
1601432    380000000
1601433    450000000
1601434     52000000
1601435    210000000
1601436    360000000
1601437    405000000
1601438    307000000
1601439    307000000
1601440    307000000
1601441    307000000
1601442    140000000
1601443    303000000
1601444    30

In [33]:
features

('year_of_completion',
 'total_household_count_in_sites',
 'apartment_building_count_in_sites',
 'supply_area',
 'total_household_count_of_area_type',
 'transaction_int_date',
 'total_parking_capacity_in_site',
 'room_count',
 'bathroom_count',
 'tallest_building_in_sites',
 'lowest_building_in_sites',
 'date_월초',
 'date_월중',
 'date_월말',
 'city_seoul',
 'city_busan',
 'floor_저층',
 'floor_중층',
 'floor_고층',
 'floor_초고층',
 'district_도심권',
 'district_동북권',
 'district_서북권',
 'district_서남권',
 'district_동남권',
 'district_중부산권',
 'district_서부산권',
 'district_동부산권')

In [0]:
# robustscaler 
from sklearn import preprocessing
scaler = preprocessing.RobustScaler()
robust_scaled_df = scaler.fit_transform(test) # array 형식 



In [0]:
robust_scaled_df = pd.DataFrame(robust_scaled_df, columns=features)

In [0]:
robust_scaled_df #train_robust_scaler 완료 
robust_scaled_df = pd.DataFrame(robust_scaled_df, columns=features)
robust_scaled_df.to_csv('./Colab Notebooks/data/robust_train.csv', index = False)


In [0]:
# train2 = pd.read_csv('./Colab Notebooks/data/robust_train.csv')  # robustscaler() 적용한 파일 내보내기, 여기서는 바로 robust_scaled_df 파일을 사용한다. 
robust_scaled_df['transaction_real_price'] = train_y


In [43]:
robust_scaled_df

,year_of_completion,total_household_count_in_sites,apartment_building_count_in_sites,supply_area,total_household_count_of_area_type,transaction_int_date,total_parking_capacity_in_site,room_count,bathroom_count,tallest_building_in_sites,...,floor_초고층,district_도심권,district_동북권,district_서북권,district_서남권,district_동남권,district_중부산권,district_서부산권,district_동부산권,transaction_real_price
0,0.000000,-0.636443,-0.574957,-0.666667,-1.2,-1.125,-1.180955,-0.503401,-2.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,215000000
1,0.000000,-0.636443,-0.574957,-0.666667,-1.2,-1.125,-1.315090,-0.625850,-1.0,-1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200000000
2,-0.583333,0.007847,-0.192504,-0.333333,-0.6,-0.500,-0.968048,0.023810,-1.0,-1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,168000000
3,-0.500000,0.007847,0.091993,-0.166667,-0.5,-0.250,-0.495413,0.306122,-1.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,165000000
4,-0.500000,0.007847,0.091993,-0.166667,-0.5,-0.250,0.783929,-0.278912,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,280000000
5,0.083333,0.411508,0.130324,0.250000,0.3,-0.375,0.218918,0.884354,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,415000000
6,0.083333,0.411508,0.130324,0.250000,0.3,-0.375,-0.589370,0.044218,0.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,267000000
7,0.500000,-0.210113,-0.298126,0.000000,0.3,-0.875,0.042392,-0.380952,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,415000000
8,0.000000,1.044464,1.252981,0.833333,0.0,-0.625,0.148371,1.299320,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,310000000
9,0.250000,2.995641,3.695911,2.750000,-0.2,-0.250,-0.539703,2.278912,0.0,-1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,319000000


## Train/Validation set 나누기 

In [0]:
robust_scaled_df['weight'] = np.ones(len(train),dtype=np.int8)
XY_all = robust_scaled_df
# train_y=train['transaction_real_price'] # 결과값
# train.drop(['transaction_real_price'], axis = 1, inplace = True)
# train.info()
mask = np.random.rand(len(train)) < 0.8
XY_train = robust_scaled_df[mask] # 훈련 0.8
XY_validate= robust_scaled_df[~mask] # 검증셋 0.2 


In [52]:
XY_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1281618 entries, 0 to 1601457
Data columns (total 30 columns):
year_of_completion                    1281618 non-null float64
total_household_count_in_sites        1281618 non-null float64
apartment_building_count_in_sites     1281618 non-null float64
supply_area                           1281618 non-null float64
total_household_count_of_area_type    1281618 non-null float64
transaction_int_date                  1281618 non-null float64
total_parking_capacity_in_site        1281618 non-null float64
room_count                            1281618 non-null float64
bathroom_count                        1281618 non-null float64
tallest_building_in_sites             1281618 non-null float64
lowest_building_in_sites              1281618 non-null float64
date_월초                               1281618 non-null float64
date_월중                               1281618 non-null float64
date_월말                               1281618 non-null float64
city_

In [57]:
# test_df=pd.read_csv('./Colab Notebooks/data/robust_test.csv')
test_df.shape
len(features)


28

In [55]:
#train_xy['transaction_real_price'] # train_y
#train_xy # ()columns = features)

test_df["weight"] = np.ones(len(test_df), dtype=np.int8)

# test_df["y"] = test_df['transaction_real_price']
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3918 entries, 0 to 3917
Data columns (total 29 columns):
year_of_completion                    3918 non-null float64
total_household_count_in_sites        3918 non-null float64
apartment_building_count_in_sites     3918 non-null float64
supply_area                           3918 non-null float64
total_household_count_of_area_type    3918 non-null float64
transaction_int_date                  3918 non-null float64
total_parking_capacity_in_site        3918 non-null float64
room_count                            3918 non-null float64
bathroom_count                        3918 non-null float64
tallest_building_in_sites             3918 non-null float64
lowest_building_in_sites              3918 non-null float64
date_월초                               3918 non-null float64
date_월중                               3918 non-null float64
date_월말                               3918 non-null float64
city_seoul                            3918 non-null f

## XGBoost

In [0]:
! pip install lightgbm

In [70]:
# XGBoost 모델을 훈련 데이터에 학습하는 코드 

# Parameter 설정

eta = 0.11 
max_depth = 7 
subsample = 0.6 
colsample_bytree = 0.85 
min_child_weight = 50 



params = {"objective":"reg:linear", 
          "booster":"gbtree" , 
          "eta": eta, 
          "max_depth": int(max_depth), 
          "subsample" : subsample, 
          "colsample_bytree": colsample_bytree, 
          "min_child_weight": min_child_weight, "silent":1,
          "colsample_bylevel": 0.7} 

X_train = XY_train.as_matrix(columns=features)
Y_train = XY_train.as_matrix(columns=["transaction_real_price"])
W_train = XY_train.as_matrix(columns=['weight'])
dtrain=xgb.DMatrix(X_train, label=Y_train, feature_names=features, weight=W_train)

X_validate = XY_validate.as_matrix(columns=features)
Y_validate = XY_validate.as_matrix(columns=['transaction_real_price'])
W_validate = XY_validate.as_matrix(columns=['weight'])
dvalidate=xgb.DMatrix(X_validate,label=Y_validate, feature_names=features, weight=W_validate) 

# XGBoost 모델을 훈련 데이터로 학습한다. 
evalist = [(dtrain,'train'),(dvalidate,'eval')]
model = xgb.train(params, dtrain, num_boost_round = 550, evals=evalist, early_stopping_rounds=5)

# 변수 중요도를 출력한다.
print('Feature Importance:')
for kv in sorted([(k,v) for k,v in model.get_fscore().items()],
                key=lambda kv:kv[1], reverse=True):
  print(kv)
  
print('model trained complete!')

# 학습한 모델을 저장한다.
import pickle

pickle.dump(model, open("./Colab Notebooks/DACON_submission/xgb.baseline0128.pkl","wb"))
best_ntree_limit = model.best_ntree_limit

[0]	train-rmse:4.63314e+08	eval-rmse:4.65163e+08
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 5 rounds.
[1]	train-rmse:4.17128e+08	eval-rmse:4.18958e+08
[2]	train-rmse:3.76537e+08	eval-rmse:3.78279e+08
[3]	train-rmse:3.41087e+08	eval-rmse:3.42767e+08
[4]	train-rmse:3.09508e+08	eval-rmse:3.11146e+08
[5]	train-rmse:2.81744e+08	eval-rmse:2.83317e+08
[6]	train-rmse:2.57537e+08	eval-rmse:2.59061e+08
[7]	train-rmse:2.36377e+08	eval-rmse:2.37855e+08
[8]	train-rmse:2.17882e+08	eval-rmse:2.19344e+08
[9]	train-rmse:2.01787e+08	eval-rmse:2.03162e+08
[10]	train-rmse:1.87756e+08	eval-rmse:1.89112e+08
[11]	train-rmse:1.75299e+08	eval-rmse:1.76644e+08
[12]	train-rmse:1.6461e+08	eval-rmse:1.65924e+08
[13]	train-rmse:1.55531e+08	eval-rmse:1.56788e+08
[14]	train-rmse:1.47841e+08	eval-rmse:1.4906e+08
[15]	train-rmse:1.40814e+08	eval-rmse:1.42001e+08
[16]	train-rmse:1.3487e+08	eval-rmse:1.36057e+08
[17]	train-rmse:1.296

In [71]:
# 모델 재학습 
X_all = XY_all.as_matrix(columns=features) 
Y_all = XY_all.as_matrix(columns=['transaction_real_price'] ) 
dall = xgb.DMatrix(X_all,label=Y_all,feature_names=features) 
evalist = [(dall,'train')] 
# 트리 개수를 늘어난 데이터 양만큼 비례해서 증가한다. 
best_ntree_limit = int(best_ntree_limit * (len(XY_train) + len(XY_validate)) / len(XY_train)) 
# XGBoost 모델 재학습 ! 
model = xgb.train(params, dall, num_boost_round=best_ntree_limit, evals=evalist) 

# 변수 중요도를 출력한다.
print('Feature Importance:')
for kv in sorted([(k,v) for k,v in model.get_fscore().items()],
                key=lambda kv:kv[1], reverse=True):
  print(kv)
  
  
# 제출을 위해 테스트 데이터에 대한 예측 값을 구한다. 
X_test = test_df.as_matrix(columns=features)
dtst = xgb.DMatrix(X_test, feature_names=features)
preds_test_xgb = model.predict(dtst, ntree_limit=best_ntree_limit)



[0]	train-rmse:4.65046e+08
[1]	train-rmse:4.19444e+08
[2]	train-rmse:3.78661e+08
[3]	train-rmse:3.42913e+08
[4]	train-rmse:3.11816e+08
[5]	train-rmse:2.83969e+08
[6]	train-rmse:2.59903e+08
[7]	train-rmse:2.38608e+08
[8]	train-rmse:2.1959e+08
[9]	train-rmse:2.03253e+08
[10]	train-rmse:1.88489e+08
[11]	train-rmse:1.75721e+08
[12]	train-rmse:1.64808e+08
[13]	train-rmse:1.55454e+08
[14]	train-rmse:1.47382e+08
[15]	train-rmse:1.4042e+08
[16]	train-rmse:1.3421e+08
[17]	train-rmse:1.29158e+08
[18]	train-rmse:1.24489e+08
[19]	train-rmse:1.20671e+08
[20]	train-rmse:1.17339e+08
[21]	train-rmse:1.14332e+08
[22]	train-rmse:1.1182e+08
[23]	train-rmse:1.09249e+08
[24]	train-rmse:1.07391e+08
[25]	train-rmse:1.05681e+08
[26]	train-rmse:1.04157e+08
[27]	train-rmse:1.03002e+08
[28]	train-rmse:1.01922e+08
[29]	train-rmse:1.01038e+08
[30]	train-rmse:1.00208e+08
[31]	train-rmse:9.93728e+07
[32]	train-rmse:9.86441e+07
[33]	train-rmse:9.80324e+07
[34]	train-rmse:9.74982e+07
[35]	train-rmse:9.69748e+07
[36]	t

In [72]:
# ! pip install lightgbm
preds_test_xgb

array([8.7251789e+08, 5.2478976e+08, 2.7047859e+08, ..., 3.5391187e+08,
       4.8488611e+08, 3.4973738e+08], dtype=float32)

## lightGBM

In [64]:

import lightgbm as lgbm 

train2 = lgbm.Dataset(XY_train[list(features)], label=XY_train['transaction_real_price'],weight=XY_train['weight'],feature_name=features)

validate2 = lgbm.Dataset(XY_validate[list(features)], label=XY_validate['transaction_real_price'],
                                 weight=XY_validate['weight'],feature_name=features, reference = train2)
param = {'task' : 'training',
    'boosting_type' : 'gbdt',
    'objective' : 'regression',
    'metric':'rmse',
    'min_data_in_leaf' : 10,
    'bagging_fraction' : 0.8,
    'verbosity' : 1,
    'num_leaves' : 70,
    'max_depth' : 7,
    'min_data_in_leaf':60,}

model = lgbm.train(param, train2, num_boost_round = 500, valid_sets =validate2, early_stopping_rounds=10)
best_iteration = model.best_iteration






[1]	valid_0's rmse: 2.99913e+08
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's rmse: 2.77364e+08
[3]	valid_0's rmse: 2.57501e+08
[4]	valid_0's rmse: 2.39848e+08
[5]	valid_0's rmse: 2.24072e+08
[6]	valid_0's rmse: 2.10392e+08
[7]	valid_0's rmse: 1.98236e+08
[8]	valid_0's rmse: 1.8727e+08
[9]	valid_0's rmse: 1.77612e+08
[10]	valid_0's rmse: 1.68981e+08
[11]	valid_0's rmse: 1.61534e+08
[12]	valid_0's rmse: 1.5493e+08
[13]	valid_0's rmse: 1.49042e+08
[14]	valid_0's rmse: 1.43765e+08
[15]	valid_0's rmse: 1.39418e+08
[16]	valid_0's rmse: 1.35416e+08
[17]	valid_0's rmse: 1.31752e+08
[18]	valid_0's rmse: 1.28129e+08
[19]	valid_0's rmse: 1.25361e+08
[20]	valid_0's rmse: 1.22854e+08
[21]	valid_0's rmse: 1.2035e+08
[22]	valid_0's rmse: 1.18403e+08
[23]	valid_0's rmse: 1.16625e+08
[24]	valid_0's rmse: 1.15014e+08
[25]	valid_0's rmse: 1.1362e+08
[26]	valid_0's rmse: 1.12141e+08
[27]	valid_0's rmse: 1.10778e+08
[28]	valid_0's rmse: 1.09778e+08
[29]	valid_0's rmse: 1.0802

In [65]:
best_iteration

500

In [66]:
# 전체 훈련 데이터에는 늘어난 양만큼 트리 개수를 늘린다. 
best_iteration = int(best_iteration * len(XY_all) / len(XY_train)) 
# 전체 훈련 데이터에 대한 LightGBM 전용 데이터를 생성한다. 
all_train = lgbm.Dataset(XY_all[list(features)], label=XY_all['transaction_real_price'],weight=XY_all['weight'], feature_name=features)
# LightGBM 모델 학습 
model2 = lgbm.train(param,all_train,num_boost_round= best_iteration)

# 변수 중요도를 출력한다.
print('Feature Importance by split:')
for kv in sorted([(k,v) for k,v in zip(features, model.feature_importance("split"))],
                key=lambda kv:kv[1], reverse=True):
  print(kv)

print('Feature Importance by gain:')
for kv in sorted([(k,v) for k,v in zip(features, model.feature_importance("gain"))],
                key=lambda kv:kv[1], reverse=True):
  print(kv)
  
# 테스트 데이터에 대한 예측 결과물을 리턴 
pred_test_lgbm = model2.predict(test_df[list(features)],num_iteration=best_iteration,verbose = 1)

Feature Importance by split:
('total_parking_capacity_in_site', 4703)
('year_of_completion', 4389)
('lowest_building_in_sites', 3863)
('total_household_count_in_sites', 3781)
('apartment_building_count_in_sites', 3559)
('total_household_count_of_area_type', 2508)
('transaction_int_date', 2346)
('room_count', 2342)
('supply_area', 2113)
('district_동북권', 1025)
('city_seoul', 629)
('district_서남권', 482)
('bathroom_count', 457)
('district_동남권', 438)
('district_도심권', 274)
('tallest_building_in_sites', 255)
('district_서부산권', 252)
('district_서북권', 151)
('floor_중층', 124)
('floor_초고층', 86)
('district_동부산권', 80)
('district_중부산권', 74)
('date_월초', 56)
('floor_고층', 46)
('date_월말', 37)
('date_월중', 27)
('city_busan', 23)
('floor_저층', 23)
Feature Importance by gain:
('total_parking_capacity_in_site', 2.2832897945145794e+23)
('district_동북권', 1.8392975291329424e+23)
('lowest_building_in_sites', 4.402875913704833e+22)
('year_of_completion', 4.094117191857559e+22)
('city_seoul', 4.081871495711427e+22)
('to

In [73]:
pred_test_lgbm

array([9.43901477e+08, 5.47819405e+08, 2.26782691e+08, ...,
       3.18457229e+08, 4.43419842e+08, 2.74779805e+08])

## 앙상블

In [0]:
# 곱셈 후 , 제곱근을 구하는 방식으로 앙상블을 수행한다 
# Y_test = np.sqrt(np.multiply(preds_test_xgb, pred_test_lgbm)) # 기하평균 
Y_test = ((0.5*preds_test_xgb)+(0.5*pred_test_lgbm)) # 단순평균 
# submission

## 제출

In [0]:
submission_file = pd.read_csv('./Colab Notebooks/data/submission.csv')
result = pd.DataFrame()
result['key'] = submission_file['key']
result['transaction_real_price']=Y_test
result.to_csv('./Colab Notebooks/DACON_submission/submission_0128.csv',index=False) 
# XGBoost and lightGBM Ensemble

In [0]:
'''
# XGBoost와 LIghtGBM을 학습하는 코드 
import os
import pickle # 사용 X 

import pandas as pd 
import numpy as np 

import xgboost as xgb
import lightgbm as lgbm 

from utils import * 


# xgboost 학습하는 함수 
def xgboost(XY_train, XY_validate, test_df, features, XY_all = train):
  eta = 0.1 
  max_depth = 6 
  subsample = 0.9 
  colsample_bytree = 0.85 
  min_child_weight = 55 
  num_boost_round = 500
  
  params = {"objective":"reg:linear", 
            "booster":"gbtree" , 
            "eta": eta, 
            "max_depth": int(max_depth), 
            "subsample" : subsample, 
            "colsample_bytree": colsample_bytree, 
            "min_child_weight": min_child_weight, "silent":1} 
  # 훈련데이터에서 X,Y,weight 추출 , as_matrix를 통해 메모리 효율적으로 array만 저장 
  X_train = XY_train.as_matrix(columns=features)
  Y_train = XY_train.as_matrix(columns=['transaction_real_price'])
  W_train = XY_train.as_matrix(columns=['weight'])
  train = xgb.DMatrix(X_train, label=Y_train, feature_names=features, weight=W_train) 
  
  X_validate = XY_validate.as_matrix(columns=features) 
  Y_validate = XY_validate.as_matrix(columns=["transaction_real_price"]) 
  W_validate = XY_validate.as_matrix(columns=["weight"]) 
  validate = xgb.DMatrix(X_validate, label=Y_validate, feature_names=features, weight=W_validate)
  
  evallist = [(train,'train'), (validate,'eval')]
  model = xgb.train(params, train, 1000, evals = evallist, early_stopping_rounds=20)
  # 학습된 모델을 저장한다 
  pickle.dump(model, open("next_multi.pickle", "wb")) # gzip을 이용하여 저장된 데이터를 압축.
  
  
  
  
  
  if XV_all is not None:
#     XY_all = train
    # 전체 훈련 데이터에 대해서 X, Y, weight 를 추출하고 , XGBoost 전용 데이터 형 태로 변환한다
    X_all = train.as_matrix(columns=features)
    Y_all = train.as_matrix(columns= ["y"]) 
    W_all = train.as_matrix(columns= "weight") 
    all_data = xgb.DMatrix(X_all, label=Y_all, feature_names=features, weight=W_all) 
    evallist = [(all_data,'all_data')] 
    # 학습할 트리 개수를 전체 훈련 데이터가 늘어난 만큼 조정한다 
    best_ntree_limit = int(best_ntree_limit * (len(XY_train) + len(XY_validate)) / len(XY_train)) 
    # 모델 학습! 
    model = xgb. train(param, all_data, best_ntree_limit, evals=evallist) 
    # 변수 중요도를 출력한다. 학습된 XGBoost 모델에서 .get_fscore()를 통해 변수 중요도를 확인할 수 있다 
  print("Feature importance:") 
  for kv in sorted([(k,v) for k,v in model.get_fscore().items()], key=lambda kv: kv[l], reverse=True): 
    print(kv) 
  # 예측에 사용할 테스트 데이터를 XGBoost 전용 데이터로 변환한다. 이 때, weight는 모두 1이기에, 별도로 작업하지 않는다 
  X_test = test_df.as_matrix(columns=features) 
  test = xgb.DMatrix(X_test,feature_names=features) 
  # 학습된 모델을 기반으로, best_ntree_limit개의 트리를 기반으로 예측한다 

  return model.predict(test,ntree_limit=best_ntree_limit) 
'''
